In [2]:
import sys
import mysql.connector
from mysql.connector import errorcode
import time
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import csv

In [3]:
def delivery_difficulty(total_boxes,total_weight,elevator_exist, delivery_floor):
    box_w = 0.6
    weight_w = 0.0003 
    elevator_y_w = 1
    elevator_n_w = 1
    floor_w = 0.15
    
    average_object_w = 3.5
    difficulty_w = 1
    #우체국 배송 운임 참고 기준
    weight_changed = ((total_weight//4000) +1)
    if weight_changed >= total_boxes:
        object_total_w = weight_changed
    else:
        object_total_w = total_boxes
    object_final_w = object_total_w / average_object_w
    
    #엘리베이터 존재 X
    if elevator_exist == 0:
        difficulty_total = object_final_w * (elevator_n_w + (delivery_floor - 1) * floor_w)
    
    #엘리베이터 존재 O
    else:
        difficulty_total = object_final_w * elevator_y_w 
    
    deli_difficulty = difficulty_total / difficulty_w
    if deli_difficulty <1:
        deli_difficulty = 1
    elif deli_difficulty>2.5:
        deli_difficulty = 2.5
    return deli_difficulty

In [7]:
#create main_invoice
try:
    conn = mysql.connector.connect(host = 'kurly-db.cxt53rg7mbem.ap-northeast-2.rds.amazonaws.com',port = 3306,database = 'kurlyDB',user = 'root',password = 'zjfflxnejans')
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor(prepared=True)
    #총 주문수
    order_num = 200
    
    for address_no in range(1,order_num+1):
        #fake invoice
        #배송지 ID(1000-1458)
        delivery_num = random.randint(1000,1459)
        sql_address = "SELECT address,region_col,region_row,floor,elevator FROM kurly_add where id = (%s);"
        cursor.execute(sql_address,[delivery_num])
        for result in cursor:
            address_name = result[0]
            address_col = result[1]
            address_row = result[2]
            floor_max = result[3]
            elevator = result[4]
        #print(address_name, address_col, address_row, floor, elevator)
        
        #상온
        ordinary_storage_weight = 0
        ordinary_storage_volume = 0
        ordinary_storage_boxes = 0
        #냉장
        cold_storage_weight = 0
        cold_storage_volume = 0
        cold_storage_boxes = 0
        #냉동
        freeze_storage_weight = 0
        freeze_storage_volume = 0
        freeze_storage_boxes = 0
        #총
        boxes_volume = 10000
        total_boxes = 0
        total_weight = 0
        order_total_quantity = random.randint(2,10)
        
        order_object_no= []
        for i in range(1,order_total_quantity):
            object_no = random.randint(1,122)
            object_quantity = random.randint(1,3)
            #print("object no:",object_no,"object_quantity",object_quantity)
            sql_object = "SELECT object_shape,object_storage,object_weight,object_volume FROM object where object_no = (%s);"
            cursor.execute(sql_object,[object_no])
            for result in cursor:
                if result[0] == 2:
                    total_boxes += (1 * object_quantity)
                    total_weight += (result[2] * object_quantity)
                else:
                    if result[1] == 0:
                        cold_storage_weight += (result[2] * object_quantity)
                        cold_storage_volume += (result[3] * object_quantity)

                    elif result[1] == 1:
                        freeze_storage_weight += (result[2] * object_quantity)
                        freeze_storage_volume += (result[3] * object_quantity)

                    else:
                        ordinary_storage_weight += (result[2] * object_quantity)
                        ordinary_storage_volume += (result[3] * object_quantity)
        if cold_storage_volume != 0:                
            cold_storage_boxes += (cold_storage_volume // boxes_volume) + 1
        if freeze_storage_volume != 0:
            freeze_storage_boxes += (freeze_storage_volume // boxes_volume) + 1
        if ordinary_storage_volume !=0:
            ordinary_storage_boxes += (ordinary_storage_volume // boxes_volume) + 1

        total_boxes = total_boxes + cold_storage_boxes + freeze_storage_boxes + ordinary_storage_boxes
        total_weight = total_weight + cold_storage_weight + freeze_storage_weight + ordinary_storage_weight
        floor= random.randint(1,floor_max)
        invoice_difficulty = round(delivery_difficulty(total_boxes, total_weight, elevator, floor),2)
        #print("배송 난이도:",invoice_difficulty)
        #print(total_boxes,"box",total_weight,"g")

        sql_invoice = "insert into fake_invoice(invoice_no,address_no,address_row,address_col,address_name,delivery_difficulty,delivery_box,delivery_weight) values (%s,%s,%s,%s,%s,%s,%s,%s);"
        cursor.execute(sql_invoice,[delivery_num+10000,delivery_num,address_row,address_col,address_name,invoice_difficulty,int(total_boxes), int(total_weight)])
    conn.commit()
    print("insert finished")

Connection established
insert finished


In [42]:
#create test_set#1
try:
    conn = mysql.connector.connect(host = 'kurly-db.cxt53rg7mbem.ap-northeast-2.rds.amazonaws.com',port = 3306,database = 'kurlyDB',user = 'root',password = 'zjfflxnejans')
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor(prepared=True)
    #물품(1-122)
    order_num = 3
    for address_no in range(1003,1020):
        sql_address = "SELECT address_name FROM address where address_no = (%s);"
        cursor.execute(sql_address,[address_no])
        for result in cursor:
            address_name = result[0]
        print(address_name)
        #상온
        ordinary_storage_weight = 0
        ordinary_storage_volume = 0
        ordinary_storage_boxes = 0
        #냉장
        cold_storage_weight = 0
        cold_storage_volume = 0
        cold_storage_boxes = 0
        #냉동
        freeze_storage_weight = 0
        freeze_storage_volume = 0
        freeze_storage_boxes = 0
        #총
        boxes_volume = 10000
        total_boxes = 0
        total_weight = 0
        order_total_quantity = random.randint(2,10)
        
        order_object_no= []
        for i in range(1,order_total_quantity):
            object_no = random.randint(1,122)
            object_quantity = random.randint(1,3)
            print("object no:",object_no,"object_quantity",object_quantity)
            sql_object = "SELECT object_shape,object_storage,object_weight,object_volume FROM object where object_no = (%s);"
            cursor.execute(sql_object,[object_no])
            for result in cursor:
                if result[0] == 2:
                    total_boxes += (1 * object_quantity)
                    total_weight += (result[2] * object_quantity)
                else:
                    if result[1] == 0:
                        cold_storage_weight += (result[2] * object_quantity)
                        cold_storage_volume += (result[3] * object_quantity)

                    elif result[1] == 1:
                        freeze_storage_weight += (result[2] * object_quantity)
                        freeze_storage_volume += (result[3] * object_quantity)

                    else:
                        ordinary_storage_weight += (result[2] * object_quantity)
                        ordinary_storage_volume += (result[3] * object_quantity)
        if cold_storage_volume != 0:                
            cold_storage_boxes += (cold_storage_volume // boxes_volume) + 1
        if freeze_storage_volume != 0:
            freeze_storage_boxes += (freeze_storage_volume // boxes_volume) + 1
        if ordinary_storage_volume !=0:
            ordinary_storage_boxes += (ordinary_storage_volume // boxes_volume) + 1

        total_boxes = total_boxes + cold_storage_boxes + freeze_storage_boxes + ordinary_storage_boxes
        total_weight = total_weight + cold_storage_weight + freeze_storage_weight + ordinary_storage_weight
        elevator = 0
        floor = 4
        invoice_difficulty = round(delivery_difficulty(total_boxes, total_weight, elevator, floor),2)
        print("배송 난이도:",invoice_difficulty)
        print(total_boxes,"box",total_weight,"g")
        sql_invoice = "insert into invoice_test(invoice_no,address_no,address_name,delivery_difficulty,delivery_box,delivery_weight,driver_region) values (%s,%s,%s,%s,%s,%s,%s);"
        cursor.execute(sql_invoice,[address_no+10000,address_no,address_name,invoice_difficulty,int(total_boxes), int(total_weight), random.randint(1,5)])
        conn.commit()

Connection established
서울 강남구 영동대로 216 하나은행 학여울역지점
object no: 48 object_quantity 2
배송 난이도: 1
1 box 400 g
서울 강남구 영동대로 50길 10 엠솔학원
object no: 29 object_quantity 3
object no: 3 object_quantity 3
object no: 59 object_quantity 2
object no: 85 object_quantity 2
object no: 7 object_quantity 2
배송 난이도: 1
2 box 3200 g
서울 강남구 영동대로 50길 14 킹즈키즈 대치점
object no: 62 object_quantity 3
object no: 13 object_quantity 3
object no: 58 object_quantity 2
배송 난이도: 1
2 box 3800 g
서울 강남구 대치동 대치우성1차아파트주택재건축 지구
object no: 45 object_quantity 2
object no: 61 object_quantity 1
object no: 17 object_quantity 1
object no: 64 object_quantity 1
배송 난이도: 1.24
3 box 2300 g
서울 강남구 영동대로 230 우성상가
object no: 75 object_quantity 2
object no: 41 object_quantity 2
object no: 63 object_quantity 2
object no: 86 object_quantity 2
object no: 37 object_quantity 1
object no: 104 object_quantity 3
object no: 111 object_quantity 2
배송 난이도: 1.66
4 box 7439 g
서울 강남구 대치동 65 대치쌍용2차아파트
object no: 24 object_quantity 1
object no: 118 object_quantity 

In [ ]:
#create test_set#1
try:
    conn = mysql.connector.connect(host = 'kurly-db.cxt53rg7mbem.ap-northeast-2.rds.amazonaws.com',port = 3306,database = 'kurlyDB',user = 'root',password = 'zjfflxnejans')
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor(prepared=True)
    #물품(1-122)
    order_num = 3
    for address_no in range(1003,1020):
        sql_address = "SELECT address_name FROM address where address_no = (%s);"
        cursor.execute(sql_address,[address_no])
        for result in cursor:
            address_name = result[0]
        print(address_name)
        #상온
        ordinary_storage_weight = 0
        ordinary_storage_volume = 0
        ordinary_storage_boxes = 0
        #냉장
        cold_storage_weight = 0
        cold_storage_volume = 0
        cold_storage_boxes = 0
        #냉동
        freeze_storage_weight = 0
        freeze_storage_volume = 0
        freeze_storage_boxes = 0
        #총
        boxes_volume = 10000
        total_boxes = 0
        total_weight = 0
        order_total_quantity = random.randint(2,10)
        
        order_object_no= []
        for i in range(1,order_total_quantity):
            object_no = random.randint(1,122)
            object_quantity = random.randint(1,3)
            print("object no:",object_no,"object_quantity",object_quantity)
            sql_object = "SELECT object_shape,object_storage,object_weight,object_volume FROM object where object_no = (%s);"
            cursor.execute(sql_object,[object_no])
            for result in cursor:
                if result[0] == 2:
                    total_boxes += (1 * object_quantity)
                    total_weight += (result[2] * object_quantity)
                else:
                    if result[1] == 0:
                        cold_storage_weight += (result[2] * object_quantity)
                        cold_storage_volume += (result[3] * object_quantity)

                    elif result[1] == 1:
                        freeze_storage_weight += (result[2] * object_quantity)
                        freeze_storage_volume += (result[3] * object_quantity)

                    else:
                        ordinary_storage_weight += (result[2] * object_quantity)
                        ordinary_storage_volume += (result[3] * object_quantity)
        if cold_storage_volume != 0:                
            cold_storage_boxes += (cold_storage_volume // boxes_volume) + 1
        if freeze_storage_volume != 0:
            freeze_storage_boxes += (freeze_storage_volume // boxes_volume) + 1
        if ordinary_storage_volume !=0:
            ordinary_storage_boxes += (ordinary_storage_volume // boxes_volume) + 1

        total_boxes = total_boxes + cold_storage_boxes + freeze_storage_boxes + ordinary_storage_boxes
        total_weight = total_weight + cold_storage_weight + freeze_storage_weight + ordinary_storage_weight
        elevator = 0
        floor = 4
        invoice_difficulty = round(delivery_difficulty(total_boxes, total_weight, elevator, floor),2)
        print("배송 난이도:",invoice_difficulty)
        print(total_boxes,"box",total_weight,"g")
        sql_invoice = "insert into invoice_test(invoice_no,address_no,address_name,delivery_difficulty,delivery_box,delivery_weight,driver_region) values (%s,%s,%s,%s,%s,%s,%s);"
        cursor.execute(sql_invoice,[address_no+10000,address_no,address_name,invoice_difficulty,int(total_boxes), int(total_weight), random.randint(1,5)])
        conn.commit()

In [48]:
#create test_set#2
try:
    conn = mysql.connector.connect(host = 'kurly-db.cxt53rg7mbem.ap-northeast-2.rds.amazonaws.com',port = 3306,database = 'kurlyDB',user = 'root',password = 'zjfflxnejans')
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor(prepared=True)
    #물품(1-122)
    order_num = 3
    for address_no in range(1070,1250):
        sql_address = "SELECT address_name FROM address where address_no = (%s);"
        cursor.execute(sql_address,[address_no])
        for result in cursor:
            address_name = result[0]
        #print(address_name)
        
        #상온
        ordinary_storage_weight = 0
        ordinary_storage_volume = 0
        ordinary_storage_boxes = 0
        #냉장
        cold_storage_weight = 0
        cold_storage_volume = 0
        cold_storage_boxes = 0
        #냉동
        freeze_storage_weight = 0
        freeze_storage_volume = 0
        freeze_storage_boxes = 0
        #총
        boxes_volume = 10000
        total_boxes = 0
        total_weight = 0
        order_total_quantity = random.randint(2,10)
        
        order_object_no= []
        for i in range(1,order_total_quantity):
            object_no = random.randint(1,122)
            object_quantity = random.randint(1,3)
            #print("object no:",object_no,"object_quantity",object_quantity)
            sql_object = "SELECT object_shape,object_storage,object_weight,object_volume FROM object where object_no = (%s);"
            cursor.execute(sql_object,[object_no])
            for result in cursor:
                if result[0] == 2:
                    total_boxes += (1 * object_quantity)
                    total_weight += (result[2] * object_quantity)
                else:
                    if result[1] == 0:
                        cold_storage_weight += (result[2] * object_quantity)
                        cold_storage_volume += (result[3] * object_quantity)

                    elif result[1] == 1:
                        freeze_storage_weight += (result[2] * object_quantity)
                        freeze_storage_volume += (result[3] * object_quantity)

                    else:
                        ordinary_storage_weight += (result[2] * object_quantity)
                        ordinary_storage_volume += (result[3] * object_quantity)
        if cold_storage_volume != 0:                
            cold_storage_boxes += (cold_storage_volume // boxes_volume) + 1
        if freeze_storage_volume != 0:
            freeze_storage_boxes += (freeze_storage_volume // boxes_volume) + 1
        if ordinary_storage_volume !=0:
            ordinary_storage_boxes += (ordinary_storage_volume // boxes_volume) + 1

        total_boxes = total_boxes + cold_storage_boxes + freeze_storage_boxes + ordinary_storage_boxes
        total_weight = total_weight + cold_storage_weight + freeze_storage_weight + ordinary_storage_weight
        elevator = 0
        floor = 4
        invoice_difficulty = round(delivery_difficulty(total_boxes, total_weight, elevator, floor),2)
        #print("배송 난이도:",invoice_difficulty)
        #print(total_boxes,"box",total_weight,"g")
        address_row = random.randint(1,6)
        address_col = random.randint(1,7)
        sql_invoice = "insert into invoice_test3(invoice_no,address_no,address_row,address_col,address_name,delivery_difficulty,delivery_box,delivery_weight) values (%s,%s,%s,%s,%s,%s,%s,%s);"
        cursor.execute(sql_invoice,[address_no+10000,address_no,address_row,address_col,address_name,invoice_difficulty,int(total_boxes), int(total_weight)])
    conn.commit()
    print("insert finished")

Connection established
insert finished
